In [ ]:
import sys
# sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/")
# sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/bib_lookup/")

import os
import pickle
import time
from copy import deepcopy
from pathlib import Path
from typing import Dict, Union, Tuple, Sequence

import numpy as np
import torch
import wfdb
from torch.nn.parallel import (  # noqa: F401
    DistributedDataParallel as DDP,
    DataParallel as DP,
)  # noqa: F401
from torch_ecg.cfg import CFG
from torch_ecg.utils.misc import str2bool
from tqdm.auto import tqdm

from cfg import TrainCfg, ModelCfg
from cfg import TrainCfg, ModelCfg
from dataset import CINC2025Dataset
from models import CRNN_CINC2025
from trainer import CINC2025Trainer

%load_ext autoreload
%autoreload 2

## Train models

In [ ]:
db_dir = Path("/home/wenh06/Jupyter/wenhao/Hot-Data/cinc2025/")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
TEST_FLAG = False

if ModelCfg.torch_dtype == torch.float64:
    torch.set_default_tensor_type(torch.DoubleTensor)
    DTYPE = np.float64
else:
    DTYPE = np.float32

CINC2025Dataset.__DEBUG__ = False
CRNN_CINC2025.__DEBUG__ = False
CINC2025Trainer.__DEBUG__ = False

In [ ]:
train_config = deepcopy(TrainCfg)
train_config.db_dir = db_dir
train_config.debug = True

train_config.n_epochs = 20
train_config.batch_size = 64  # 16G (Tesla T4)
# train_config.log_step = 20
train_config.learning_rate = 1e-4  # 5e-4, 1e-3
train_config.lr = train_config.learning_rate
train_config.max_lr = 6e-4
train_config.early_stopping.patience = train_config.n_epochs // 3

# augmentations configurations
# TODO: add augmentation configs

model_config = deepcopy(ModelCfg)

In [ ]:
model = CRNN_CINC2025(config=model_config)
model = model.to(device=DEVICE)

if isinstance(model, DP):
    print("model size:", model.module.module_size, model.module.module_size_)
    print("Using devices:", model.device_ids)
else:
    print("model size:", model.module_size, model.module_size_)
    print("Using device:", model.device)

In [ ]:
ds_train = CINC2025Dataset(train_config, training=True, lazy=True)
ds_val = CINC2025Dataset(train_config, training=False, lazy=True)

In [ ]:
print(f"train size: {len(ds_train)}, val size: {len(ds_val)}")

In [ ]:
trainer = CINC2025Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=DEVICE,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
best_model_state_dict = trainer.train()

In [ ]:
trainer.log_manager.flush()
trainer.log_manager.close()

In [ ]:
del trainer, model, best_model_state_dict
torch.cuda.empty_cache()

## Debug

In [ ]:
for idx, input_data in tqdm(enumerate(ds_val), total=len(ds_val)):
    pass

## Results analyze and visualize

In [ ]:
df_results = pd.read_csv("./results/TorchECG_02-20_16-47_CRNN_CINC2025_adamw_amsgrad_LR_0.0001_BS_64.csv")

In [ ]:
df_val = df_results[df_results.part=="val"]
df_train = df_results[df_results.part=="train"]

In [ ]:
plt.plot(df_train.challenge_score.dropna())
plt.plot(df_val.challenge_score.dropna())